In [1]:
import dash
import more_itertools
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import re

In [2]:
def set_groupings(data, mode):
    groupings = {} 
    for i,col in enumerate(data.row_description):
        if type(col) is str:
            x=re.search(r'(group[0-9]+)$',col)
            if x:
                temp=list(data.data[str(data.row_description.index[i])].unique())
                if mode == 'indiv':
                    temp.append('-')
                groupings.update({str(data.row_description.index[i]):temp})
    return groupings

class StatsStructure:
       def __init__(self,row_idx:pd.Series,row_description:pd.Series,data: pd.DataFrame):
        self.row_idx = row_idx
        self.row_description = row_description
        self.data = data
        
class DataStructure:
       def __init__(self,row_idx:pd.Series,row_description:pd.Series,mode:str,data: pd.DataFrame,groupings:dict):
        self.row_idx = row_idx
        self.row_description = row_description
        self.data = data
        self.mode = mode
        self.groupings = groupings
        
def load_data(path,mode):
    
    data = pd.read_csv(path,delimiter='\t',low_memory=False,header=1)
    data_col_idx=data.iloc[0]
    data_comments=data.iloc[1]
    data = data.drop([data.index[0],data.index[1]]).reset_index()
    
    myData=DataStructure(row_idx = data_col_idx,row_description = data_comments,data = data,mode=mode,groupings=None) # Create Data (indiv or Group) class

    #find the groupings within the data
    groupings=set_groupings(myData, mode)
    
    myData.groupings = groupings
        
    return myData

def load_stats(path):
    stats = pd.read_csv(path,delimiter='\t',low_memory=False) 
    stats_col_idx=stats.iloc[0]
    stats_comments=stats.iloc[1]
    stats = stats.drop([stats.index[0],stats.index[1]]).reset_index()
    
    myStats = StatsStructure(row_idx = stats_col_idx,row_description = stats_comments,data = stats)
    #assign data to it
    
    return myStats

In [3]:
Group_Stats = load_stats('/Volumes/dusom_civm-kjh60/All_Staff/18.gaj.42/Scalar_and_Volume/Main_Effects_2025_01_14_NoB6/Non_Erode/Bilateral/Group_Statistical_Results_Age_Class_Strain_Sex.csv')
Group_Data = load_data('/Volumes/dusom_civm-kjh60/All_Staff/18.gaj.42/Scalar_and_Volume/Main_Effects_2025_01_14_NoB6/Non_Erode/Bilateral/Group_Data_Table_Age_Class_Strain_Sex.csv',mode='group')
Indiv_Data = load_data('/Volumes/dusom_civm-kjh60/All_Staff/18.gaj.42/Scalar_and_Volume/Main_Effects_2025_01_14_NoB6/Non_Erode/Subject_Data_Table.csv',mode='indiv')

In [34]:
Group_Stats.data['source_of_variation']

0       Age_Class
1          Strain
2             Sex
3       Age_Class
4          Strain
          ...    
4153       Strain
4154          Sex
4155    Age_Class
4156       Strain
4157          Sex
Name: source_of_variation, Length: 4158, dtype: object

In [5]:
Group_Data.data

,index,# Strain,Sex,Age_Class,ROI,Structure,hemisphere_assignment,GN_Symbol,ARA_abbrev,id64_fSABI,...,rd_mean_group_median,md_mean_group_median,volume_fraction_group_median,volume_mm3_group_IQR,fa_mean_group_IQR,ad_mean_group_IQR,rd_mean_group_IQR,md_mean_group_IQR,volume_fraction_group_IQR,stratification
0,2,-,F,-,20014,DTG__Dorsal_tegmental_nucleus,0,DTG-B,DTN,09112272600539590634,...,0.296894,0.314611,0.000216,0.011300,0.012032,0.028183,0.021270,0.023801,0.000017,hemisphere=bilateral
1,3,-,F,-,20061,LAA__Lateral_amygdala,0,LAA-B,LA,09310081833820345102,...,0.293061,0.329091,0.001798,0.080805,0.025600,0.037819,0.027743,0.030202,0.000205,hemisphere=bilateral
2,4,-,F,-,20108,MBB__Midbrain_behavioral_state_related,0,MBB-B,MBsta,15919898350782410547,...,0.245492,0.281820,0.002863,0.152726,0.020122,0.031194,0.021170,0.025395,0.000313,hemisphere=bilateral
3,5,-,F,-,20370,ORB__Orbital_cortex,0,ORB-B,ORB,12248104564192473910,...,0.300275,0.334720,0.010667,0.530985,0.015830,0.040477,0.031451,0.035050,0.001408,hemisphere=bilateral
4,6,-,F,-,20476,CA1__CA1_of_hippocampus,0,CA1-B,CA1,12752956421510536081,...,0.300484,0.337507,0.021338,0.828258,0.015529,0.035833,0.024864,0.031609,0.001350,hemisphere=bilateral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28639,28641,BXD89,M,Young,29716,VTA__Ventral_tegmental_area,0,VTA-B,VTA,16999847868957461500,...,0.243074,0.273476,0.000996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,hemisphere=bilateral
28640,28642,BXD89,M,Young,29734,VCP__Visual_cortex_primary,0,VCP-B,VISp,04787926488316141785,...,0.288463,0.307188,0.012740,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,hemisphere=bilateral
28641,28643,BXD89,M,Young,29823,cng__cingulum,0,cng-B,cing,00806252043942724490,...,0.186563,0.231115,0.001292,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,hemisphere=bilateral
28642,28644,BXD89,M,Young,29828,MBO__Mammillary_body,0,MBO-B,MBO,00192114956764024685,...,0.269243,0.292905,0.002313,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,hemisphere=bilateral


In [6]:
data_pivot = pd.pivot_table(Group_Data.data, values='fa_mean_group_mean', index='GN_Symbol', columns=list(Group_Data.groupings.keys()))

In [25]:
data_pivot['-']['-']['Old']

GN_Symbol
ACC-B    0.179579
ADT-B    0.168496
AHY-B    0.108138
AIC-B    0.172823
AMT-B    0.214327
           ...   
otc-B    0.614896
pcm-B    0.294198
scp-B    0.354972
stm-B    0.567393
wmt-B    0.468745
Name: Old, Length: 231, dtype: float64

In [8]:
list(list(Group_Data.groupings.keys())

SyntaxError: incomplete input (3753572177.py, line 1)

In [18]:
import math

n = 4  # Total number of items
k = 2   # Number of items to choose

result = math.comb(n, k)

In [19]:
result

6

In [20]:
Group_Data.groupings

{'# Strain': ['-',
  'BXD101',
  'BXD24',
  'BXD29',
  'BXD34',
  'BXD43',
  'BXD44',
  'BXD48a',
  'BXD51',
  'BXD60',
  'BXD62',
  'BXD65b',
  'BXD77',
  'BXD89'],
 'Sex': ['F', 'M', '-'],
 'Age_Class': ['-', 'Old', 'Young']}

In [21]:
Indiv_Data.groupings

{'Strain': ['BXD62',
  'BXD65b',
  'BXD44',
  'BXD89',
  'BXD24',
  'BXD29',
  'BXD48a',
  'BXD51',
  'BXD43',
  'BXD34',
  'BXD60',
  'BXD101',
  'BXD77',
  '-'],
 'Sex': ['F', 'M', '-'],
 'Age_Class': ['Young', 'Old', '-'],
 'Perfusion_at': ['Duke', 'UTHSC', '-'],
 'Scanner': ['A', '-']}

'-'

In [28]:
bin('1')

TypeError: 'str' object cannot be interpreted as an integer

In [30]:
bin(1)

'0b1'

In [31]:
len(config.groups_to_include)

NameError: name 'config' is not defined

In [39]:
dcc.list(Group_Stats.data['source_of_variation'].unique())


['Age_Class', 'Strain', 'Sex']

In [42]:
    sov=list(Group_Stats.data['source_of_variation'].unique())
    contrast=list(Group_Stats.data['contrast'].unique())

In [48]:
list(Group_Stats.data.columns)

['index',
 '# ROI',
 'Structure',
 'hemisphere_assignment',
 'GN_Symbol',
 'ARA_abbrev',
 'id64_fSABI',
 'id32_fSABI',
 'structure_id',
 'contrast',
 'study_model',
 'statistical_test',
 'source_of_variation',
 'sum_of_squares',
 'df',
 'F_Statistic',
 'Number_of_Groupings',
 'pval',
 'eta2',
 'H2RI',
 'cohenF',
 'stratification',
 'pval_BH',
 'cohenD_Young - -_Old - -',
 'cohenD_Young - F_Old - F',
 'cohenD_Young - M_Old - M',
 'cohenD_Young BXD101 -_Old BXD101 -',
 'cohenD_Young BXD24 -_Old BXD24 -',
 'cohenD_Young BXD29 -_Old BXD29 -',
 'cohenD_Young BXD34 -_Old BXD34 -',
 'cohenD_Young BXD43 -_Old BXD43 -',
 'cohenD_Young BXD44 -_Old BXD44 -',
 'cohenD_Young BXD48a -_Old BXD48a -',
 'cohenD_Young BXD51 -_Old BXD51 -',
 'cohenD_Young BXD60 -_Old BXD60 -',
 'cohenD_Young BXD62 -_Old BXD62 -',
 'cohenD_Young BXD65b -_Old BXD65b -',
 'cohenD_Young BXD77 -_Old BXD77 -',
 'cohenD_Young BXD89 -_Old BXD89 -',
 'absolute_error_Young - -_Old - -',
 'absolute_error_Young - F_Old - F',
 'absol

In [47]:
Indiv_Data.groupings

{'Strain': ['BXD62',
  'BXD65b',
  'BXD44',
  'BXD89',
  'BXD24',
  'BXD29',
  'BXD48a',
  'BXD51',
  'BXD43',
  'BXD34',
  'BXD60',
  'BXD101',
  'BXD77',
  '-'],
 'Sex': ['F', 'M', '-'],
 'Age_Class': ['Young', 'Old', '-'],
 'Perfusion_at': ['Duke', 'UTHSC', '-'],
 'Scanner': ['A', '-']}

In [56]:
slider_input=list(Group_Stats.data['source_of_variation'].unique())

slider_dict={}
for i in range(len(slider_input)):
    slider_dict.update({i:slider_input[i]})



In [57]:
slider_dict

{0: 'Age_Class', 1: 'Strain', 2: 'Sex'}